# ***K Nearest Neighbours***

In [11]:
import pandas as pd
from sklearn.metrics import precision_score, accuracy_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import seaborn as sns

In [16]:
df = sns.load_dataset('iris')
X = df.drop('species', axis=1)
y = df['species']

In [17]:
le = LabelEncoder()
y = le.fit_transform(y)

In [18]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train, y_train)

y_pred = knn_classifier.predict(X_test)

print("recall score: ", recall_score(y_test, y_pred, average='macro'))
print("accuracy score: ", accuracy_score(y_test, y_pred,))
print("precision score: ", precision_score(y_test, y_pred, average='macro'))

recall score:  1.0
accuracy score:  1.0
precision score:  1.0


In [34]:
from sklearn.model_selection import GridSearchCV

# classifier = KNeighborsClassifier()
# param_grid = {"n_neighbors": [3, 5, 7, 9]}

classifierCV = GridSearchCV(
    KNeighborsClassifier(),
    {"n_neighbors": [3, 5, 7, 9,]},
    cv=5,
    scoring={
        "accuracy": "accuracy",
        "precision": "precision_macro",
        "recall": "recall_macro"
    },
    refit="accuracy"
)
classifierCV.fit(X_train, y_train)

y_pred = classifierCV.predict(X_test)

print("recall score: ", recall_score(y_test, y_pred, average='macro'))
print("accuracy score: ", accuracy_score(y_test, y_pred,))
print("precision score: ", precision_score(y_test, y_pred, average='macro'))

print("Best parameters found: ", classifierCV.best_params_)
print(classifierCV.cv_results_["mean_test_accuracy"])

recall score:  1.0
accuracy score:  1.0
precision score:  1.0
Best parameters found:  {'n_neighbors': 3}
[0.95       0.93333333 0.94166667 0.94166667]


In [53]:
df = sns.load_dataset('titanic')
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [61]:
df['age'].fillna(df['age'].mean(), inplace=True)
df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)
df.drop('deck', axis=1, inplace=True)

C:\Users\Abid\AppData\Local\Temp\ipykernel_3512\2321273515.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['age'].fillna(df['age'].mean(), inplace=True)
C:\Users\Abid\AppData\Local\Temp\ipykernel_3512\2321273515.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [62]:
X = df[['pclass','sex','age','sibsp','parch','fare','embarked','class']]
y = df['survived']


In [63]:
cat_col = X.select_dtypes(include=['object']).columns
num_col = X.select_dtypes(include=['int64','float64']).columns


In [64]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


preprocessing = ColumnTransformer(
    transformers=[
                #   ('num_imputer', SimpleImputer(strategy="mean"), num_col),
                #   ('cat_imputer', SimpleImputer(strategy="most_frequent"), cat_col),
                  ('numerical', StandardScaler(), num_col),
                  ('categorical', OneHotEncoder(drop='first'), cat_col)
                  ])
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('knn', KNeighborsClassifier())
])
# X = pd.get_dummies(X, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {"knn__n_neighbors": [3, 5, 7, 9]}

classifierCV = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="recall"
)

classifierCV.fit(X_train, y_train)

y_pred = classifierCV.predict(X_test)

print("recall score: ", recall_score(y_test, y_pred))
print("accuracy score: ", accuracy_score(y_test, y_pred))
print("precision score: ", precision_score(y_test, y_pred))

print(classifierCV.best_params_)

recall score:  0.7567567567567568
accuracy score:  0.8268156424581006
precision score:  0.8115942028985508
{'knn__n_neighbors': 3}
